In [2]:
%pip install polars
%pip install duckdb

  Using cached polars-1.35.1-py3-none-any.whl.metadata (10 kB)
  Using cached polars_runtime_32-1.35.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.5 kB)
Using cached polars-1.35.1-py3-none-any.whl (783 kB)
Using cached polars_runtime_32-1.35.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (41.3 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
  Using cached duckdb-1.4.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (14 kB)
Using cached duckdb-1.4.1-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (20.5 MB)

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import polars as pl

# Load your raw notes dataset (the pre-aggregated one)
path = "/home/jovyan/Shared/2025-09-27-input/notes-00000.parquet"
df = pl.read_parquet(path)

df.head()


Unique classification values (printed one per line):

• 'MISINFORMED_OR_POTENTIALLY_MISLEADING'
• 'NOT_MISLEADING'


In [11]:
import polars as pl
import os
from datetime import datetime

# =========================================================
# CONFIG
# =========================================================
INPUT_PATH  = "/home/jovyan/Shared/2025-09-27-input/notes-00000.parquet"
OUTPUT_DIR  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core"
OUTPUT_NAME = "notes_writers_2023_aggregated.parquet"

CUTOFF_DATE = datetime(2023, 1, 1)

# =========================================================
# PIPELINE
# =========================================================
print("🚀 Cleaning + aggregating Community Notes (2023+) ...")

df = pl.scan_parquet(INPUT_PATH)

# --- Rename to consistent naming ---
df = df.rename({
    "noteAuthorParticipantId": "user_id",   # ✅ your schema key
    "createdAtMillis": "created_at_ms"
})

# --- Drop deprecated + metadata columns ---
drop_cols = [
    "believable", "harmful", "validationDifficulty",
    "_processing_commit_hash", "_processed_at", "_data_date"
]
df = df.drop([c for c in drop_cols if c in df.collect_schema().names()])

# --- Convert ms → datetime ---
df = df.with_columns([
    pl.from_epoch(pl.col("created_at_ms"), time_unit="ms").alias("created_at")
])

# --- Filter to 2023+ ---
df = df.filter(pl.col("created_at") >= pl.lit(CUTOFF_DATE))

# --- Truncate to 14-day windows ---
df = df.with_columns([
    pl.col("created_at").dt.truncate("14d").alias("period_start")
])

# --- Identify binary (checkbox) columns dynamically ---
main_cols = [
    "noteId", "tweetId", "user_id", "created_at", "created_at_ms",
    "summary", "classification", "period_start"
]
binary_cols = [c for c in df.collect_schema().names() if c not in main_cols]

# --- Aggregation expressions ---
agg_exprs = [
    pl.count().alias("notes_written"),
    (pl.col("classification") == "MISINFORMED_OR_POTENTIALLY_MISLEADING")
        .sum().alias("num_misleading"),
    (pl.col("classification") == "NOT_MISLEADING")
        .sum().alias("num_not_misleading"),
    (pl.col("trustworthySources") == 1).mean().alias("avg_trustworthySources"),
    pl.col("isMediaNote").mean().alias("media_note_ratio"),
]

# --- Summed reasoning flags ---
misleading_cols = [c for c in binary_cols if c.startswith("misleading")]
not_misleading_cols = [c for c in binary_cols if c.startswith("notMisleading")]

if misleading_cols:
    agg_exprs.append(
        pl.sum_horizontal([pl.col(c) for c in misleading_cols]).alias("misleading_flag_sum")
    )
if not_misleading_cols:
    agg_exprs.append(
        pl.sum_horizontal([pl.col(c) for c in not_misleading_cols]).alias("not_misleading_flag_sum")
    )

# --- Perform aggregation ---
agg = (
    df.group_by(["user_id", "period_start"])
      .agg(agg_exprs)
      .with_columns([
          (pl.col("num_misleading") / pl.col("notes_written")).alias("share_misleading"),
          ((pl.col("misleading_flag_sum") + pl.col("not_misleading_flag_sum")) /
           pl.col("notes_written")).alias("avg_flags_per_note")
      ])
      .sort(["user_id", "period_start"])
)

# --- Execute + Save ---
result = agg.collect()
out_path = os.path.join(OUTPUT_DIR, OUTPUT_NAME)
result.write_parquet(out_path)

print(f"✅ Aggregation complete — saved to:\n{out_path}")
print(f"Rows: {result.height:,} | Columns: {len(result.columns)}")
result.head(5)



🚀 Cleaning + aggregating Community Notes (2023+) ...


/tmp/ipykernel_274/3661600488.py:56: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  pl.count().alias("notes_written"),


✅ Aggregation complete — saved to:
/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/notes_writers_2023_aggregated.parquet
Rows: 1,028,703 | Columns: 11


user_id,period_start,notes_written,num_misleading,num_not_misleading,avg_trustworthySources,media_note_ratio,misleading_flag_sum,not_misleading_flag_sum,share_misleading,avg_flags_per_note
str,datetime[ms],u32,u32,u32,f64,f64,list[i64],list[i64],f64,list[f64]
"""000045A5FA0CF004F68CBF2913506C…",2023-03-30 00:00:00,5,5,0,1.0,0.0,"[2, 1, … 1]","[0, 0, … 0]",1.0,"[0.4, 0.2, … 0.2]"
"""000045A5FA0CF004F68CBF2913506C…",2023-04-27 00:00:00,1,1,0,1.0,0.0,[2],[0],1.0,[2.0]
"""000045A5FA0CF004F68CBF2913506C…",2023-05-11 00:00:00,1,0,1,1.0,0.0,[0],[1],0.0,[1.0]
"""000045A5FA0CF004F68CBF2913506C…",2023-06-08 00:00:00,1,1,0,1.0,0.0,[2],[0],1.0,[2.0]
"""000045A5FA0CF004F68CBF2913506C…",2023-08-31 00:00:00,1,1,0,0.0,0.0,[3],[0],1.0,[3.0]


In [19]:
import polars as pl
import os
from datetime import datetime

# =========================================================
# CONFIG
# =========================================================
INPUT_PATH  = "/home/jovyan/Shared/2025-09-27-input/noteStatusHistory-00000.parquet"
OUTPUT_DIR  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core"
OUTPUT_NAME = "note_status_history_2023_aggregated.parquet"

CUTOFF_DATE = datetime(2023, 1, 1)

# =========================================================
# PIPELINE
# =========================================================
print("🚀 Cleaning + aggregating Note Status History (2023+) ...")

df = pl.scan_parquet(INPUT_PATH)

# --- Rename to consistent naming ---
df = df.rename({
    "noteAuthorParticipantId": "user_id",
    "createdAtMillis": "created_at_ms",
    "mostRecentNonNMRStatus": "latestNonNMRStatus",
})

# --- Drop metadata columns ---
drop_cols = [
    "timestampMinuteOfFinalScoringOutput",
    "timestampMillisOfRetroLock",
    "timestampMillisOfNmrDueToMinStableCrhTime",
    "timestampMillisOfFirstNmrDueToMinStableCrhTime",
    "_processing_commit_hash",
    "_processed_at",
    "_data_date",
]
df = df.drop([c for c in drop_cols if c in df.collect_schema().names()])

# --- Ensure timestamp columns are numeric ---
timestamp_cols = [
    "created_at_ms",
    "timestampMillisOfFirstNonNMRStatus",
    "timestampMillisOfStatusLock",
]

for col in timestamp_cols:
    if col in df.collect_schema().names():
        df = df.with_columns(pl.col(col).cast(pl.Int64, strict=False))


# --- Convert ms → datetime ---
df = df.with_columns([
    pl.from_epoch(pl.col("created_at_ms"), time_unit="ms").alias("created_at")
])

# --- Filter to 2023+ only ---
df = df.filter(pl.col("created_at") >= pl.lit(CUTOFF_DATE))

# --- Truncate to 14-day windows ---
df = df.with_columns([
    pl.col("created_at").dt.truncate("14d").alias("period_start")
])

# --- Aggregations per user × period ---
agg_exprs = [
    pl.len().alias("notes_with_status"),
    (pl.col("currentStatus") == "CURRENTLY_RATED_HELPFUL")
        .sum().alias("notes_helpful"),
    (pl.col("currentStatus") == "CURRENTLY_RATED_NOT_HELPFUL")
        .sum().alias("notes_not_helpful"),
    (pl.col("currentStatus") == "NEEDS_MORE_RATINGS")
        .sum().alias("notes_nmr"),
    (pl.col("lockedStatus") != "").sum().alias("notes_locked"),
    (pl.col("firstNonNMRStatus") == "CURRENTLY_RATED_HELPFUL")
        .sum().alias("first_helpful"),
    (pl.col("latestNonNMRStatus") == "CURRENTLY_RATED_HELPFUL")
        .sum().alias("latest_helpful"),
    (pl.col("latestNonNMRStatus") == "CURRENTLY_RATED_NOT_HELPFUL")
        .sum().alias("latest_not_helpful"),

    # Timing metrics (average days)
    ((pl.col("timestampMillisOfFirstNonNMRStatus") - pl.col("created_at_ms")) / (1000 * 60 * 60 * 24))
        .mean().alias("avg_days_to_first_nonNMR"),
    ((pl.col("timestampMillisOfStatusLock") - pl.col("created_at_ms")) / (1000 * 60 * 60 * 24))
        .mean().alias("avg_days_to_lock"),
]

# --- Compute ratios ---
agg = (
    df.group_by(["user_id", "period_start"])
      .agg(agg_exprs)
      .with_columns([
          (pl.col("notes_helpful") / pl.col("notes_with_status")).alias("share_helpful"),
          (pl.col("notes_not_helpful") / pl.col("notes_with_status")).alias("share_not_helpful"),
      ])
      .sort(["user_id", "period_start"])
)

# --- Execute & save ---
result = agg.collect()
out_path = os.path.join(OUTPUT_DIR, OUTPUT_NAME)
result.write_parquet(out_path)

print(f"✅ Aggregation complete — saved to:\n{out_path}")
print(f"Rows: {result.height:,} | Columns: {len(result.columns)}")
result.head(5)



🚀 Cleaning + aggregating Note Status History (2023+) ...
✅ Aggregation complete — saved to:
/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_status_history_2023_aggregated.parquet
Rows: 1,089,163 | Columns: 14


user_id,period_start,notes_with_status,notes_helpful,notes_not_helpful,notes_nmr,notes_locked,first_helpful,latest_helpful,latest_not_helpful,avg_days_to_first_nonNMR,avg_days_to_lock,share_helpful,share_not_helpful
str,datetime[ms],u32,u32,u32,u32,u32,u32,u32,u32,f64,f64,f64,f64
"""000045A5FA0CF004F68CBF2913506C…",2023-03-30 00:00:00,7,2,0,5,7,2,2,1,0.218733,29.193681,0.285714,0.0
"""000045A5FA0CF004F68CBF2913506C…",2023-04-27 00:00:00,1,0,0,1,1,0,0,0,null,14.00007,0.0,0.0
"""000045A5FA0CF004F68CBF2913506C…",2023-05-11 00:00:00,1,0,0,1,1,0,0,0,null,14.007234,0.0,0.0
"""000045A5FA0CF004F68CBF2913506C…",2023-06-08 00:00:00,1,0,0,1,1,0,0,0,null,14.032289,0.0,0.0
"""000045A5FA0CF004F68CBF2913506C…",2023-08-31 00:00:00,1,0,0,1,1,0,0,0,null,14.011199,0.0,0.0


In [20]:
import polars as pl
import os
from datetime import datetime

# =========================================================
# CONFIG
# =========================================================
INPUT_PATH  = "/home/jovyan/Shared/2025-09-27-input/userEnrollment-00000.parquet"
OUTPUT_DIR  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core"
OUTPUT_NAME = "user_enrollment_2023_aggregated.parquet"

CUTOFF_DATE = datetime(2023, 1, 1)

# =========================================================
# PIPELINE
# =========================================================
print("🚀 Cleaning + aggregating User Enrollment (2023+) ...")

df = pl.scan_parquet(INPUT_PATH)

# --- Rename for consistency ---
df = df.rename({"participantId": "user_id"})

# --- Drop metadata columns ---
drop_cols = ["_processing_commit_hash", "_processed_at", "_data_date"]
df = df.drop([c for c in drop_cols if c in df.collect_schema().names()])

# --- Ensure numeric timestamps ---
timestamp_cols = ["timestampOfLastStateChange", "timestampOfLastEarnOut"]
for col in timestamp_cols:
    if col in df.collect_schema().names():
        df = df.with_columns(pl.col(col).cast(pl.Int64, strict=False))

# --- Convert timestamps to datetime (for filtering & binning) ---
df = df.with_columns([
    pl.from_epoch(pl.col("timestampOfLastStateChange"), time_unit="ms").alias("last_state_change_dt")
])

# --- Filter to users active after 2023 ---
df = df.filter(pl.col("last_state_change_dt") >= pl.lit(CUTOFF_DATE))

# --- Create 14-day period bins from last_state_change_dt ---
df = df.with_columns([
    pl.col("last_state_change_dt").dt.truncate("14d").alias("period_start")
])

# --- Derived flags ---
df = df.with_columns([
    (pl.col("enrollmentState") == "newUser").cast(pl.Int8).alias("is_new_user"),
    (pl.col("enrollmentState") == "earnedIn").cast(pl.Int8).alias("is_earned_in"),
    (pl.col("enrollmentState") == "atRisk").cast(pl.Int8).alias("is_at_risk"),
    (pl.col("enrollmentState").str.contains("earnedOut")).cast(pl.Int8).alias("is_earned_out"),
    (pl.col("modelingPopulation") == "CORE").cast(pl.Int8).alias("is_core_population"),
    (pl.col("timestampOfLastEarnOut") != 1).cast(pl.Int8).alias("has_ever_earned_out")
])

# --- Compute days since state changes ---
current_time = pl.lit(datetime.now().timestamp() * 1000)
df = df.with_columns([
    ((current_time - pl.col("timestampOfLastStateChange")) / (1000 * 60 * 60 * 24))
        .alias("days_since_last_state_change"),
    (
        pl.when(pl.col("timestampOfLastEarnOut") != 1)
        .then((current_time - pl.col("timestampOfLastEarnOut")) / (1000 * 60 * 60 * 24))
        .otherwise(None)
        .alias("days_since_last_earnout")
    )
])

# --- Aggregate per user × period ---
agg = (
    df.group_by(["user_id", "period_start"])
      .agg([
          pl.len().alias("records"),  # modern replacement for count()
          pl.mean("successfulRatingNeededToEarnIn").alias("avg_successfulRatingNeededToEarnIn"),
          pl.mean("days_since_last_state_change").alias("avg_days_since_state_change"),
          pl.mean("days_since_last_earnout").alias("avg_days_since_earnout"),
          pl.max("is_new_user").alias("is_new_user"),
          pl.max("is_earned_in").alias("is_earned_in"),
          pl.max("is_at_risk").alias("is_at_risk"),
          pl.max("is_earned_out").alias("is_earned_out"),
          pl.max("has_ever_earned_out").alias("has_ever_earned_out"),
          pl.max("is_core_population").alias("is_core_population"),
          pl.mean("modelingGroup").alias("avg_modelingGroup"),
      ])
      .sort(["user_id", "period_start"])
)

# --- Execute + Save ---
result = agg.collect()
out_path = os.path.join(OUTPUT_DIR, OUTPUT_NAME)
result.write_parquet(out_path)

print(f"✅ Aggregation complete — saved to:\n{out_path}")
print(f"Rows: {result.height:,} | Columns: {len(result.columns)}")
result.head(5)


🚀 Cleaning + aggregating User Enrollment (2023+) ...
✅ Aggregation complete — saved to:
/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/user_enrollment_2023_aggregated.parquet
Rows: 1,279,984 | Columns: 13


user_id,period_start,records,avg_successfulRatingNeededToEarnIn,avg_days_since_state_change,avg_days_since_earnout,is_new_user,is_earned_in,is_at_risk,is_earned_out,has_ever_earned_out,is_core_population,avg_modelingGroup
str,datetime[ms],u32,f64,f64,f64,i8,i8,i8,i8,i8,i8,f64
"""0000010BB832A9CFDF102BF7B66896…",2024-03-28 00:00:00,1,5.0,583.021264,null,1,0,0,0,0,1,6.0
"""000011269AD6F327AED0F4086A732B…",2024-03-28 00:00:00,1,5.0,583.021264,null,0,1,0,0,0,1,3.0
"""00001E2644DAE39EE4C52C373B921D…",2025-07-31 00:00:00,1,5.0,96.346167,null,1,0,0,0,0,1,13.0
"""00002C7FD6E0080A69D0AB879C3D9B…",2025-08-28 00:00:00,1,5.0,61.836681,null,0,1,0,0,0,1,1.0
"""0000315D36021A528D85155729DDBF…",2024-10-10 00:00:00,1,5.0,380.346775,null,1,0,0,0,0,1,13.0


In [21]:
import polars as pl
import os
from datetime import datetime

# =========================================================
# CONFIG
# =========================================================
INPUT_PATH  = "/home/jovyan/Shared/2025-09-27-input/noteRequests-00000.parquet"
OUTPUT_DIR  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core"
OUTPUT_NAME = "note_requests_2023_aggregated.parquet"

CUTOFF_DATE = datetime(2023, 1, 1)

# =========================================================
# PIPELINE
# =========================================================
print("🚀 Cleaning + aggregating Note Requests (2023+) ...")

df = pl.scan_parquet(INPUT_PATH)

# --- Rename for consistency ---
df = df.rename({
    "userId": "user_id",
    "createdAtMillis": "created_at_ms"
})

# --- Drop metadata columns if present ---
drop_cols = ["_processing_commit_hash", "_processed_at", "_data_date"]
df = df.drop([c for c in drop_cols if c in df.collect_schema().names()])

# --- Ensure numeric timestamp ---
if "created_at_ms" in df.collect_schema().names():
    df = df.with_columns(pl.col("created_at_ms").cast(pl.Int64, strict=False))

# --- Convert to datetime ---
df = df.with_columns([
    pl.from_epoch(pl.col("created_at_ms"), time_unit="ms").alias("created_at")
])

# --- Filter to 2023+ ---
df = df.filter(pl.col("created_at") >= pl.lit(CUTOFF_DATE))

# --- 14-day bins ---
df = df.with_columns([
    pl.col("created_at").dt.truncate("14d").alias("period_start")
])

# --- Derived flags ---
df = df.with_columns([
    (pl.col("sourceLink").str.strip_chars().is_not_null() & (pl.col("sourceLink") != ""))
        .cast(pl.Int8)
        .alias("has_source_link")
])

# --- Aggregations per user × period ---
agg = (
    df.group_by(["user_id", "period_start"])
      .agg([
          pl.len().alias("notes_requested"),
          pl.n_unique("tweetId").alias("unique_tweets_requested"),
          pl.sum("has_source_link").alias("with_source_link")
      ])
      .with_columns([
          (pl.col("with_source_link") / pl.col("notes_requested")).alias("share_with_link")
      ])
      .sort(["user_id", "period_start"])
)

# --- Execute + Save ---
result = agg.collect()
out_path = os.path.join(OUTPUT_DIR, OUTPUT_NAME)
result.write_parquet(out_path)

print(f"✅ Aggregation complete — saved to:\n{out_path}")
print(f"Rows: {result.height:,} | Columns: {len(result.columns)}")
result.head(5)


🚀 Cleaning + aggregating Note Requests (2023+) ...
✅ Aggregation complete — saved to:
/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_requests_2023_aggregated.parquet
Rows: 3,817,344 | Columns: 6


user_id,period_start,notes_requested,unique_tweets_requested,with_source_link,share_with_link
str,datetime[ms],u32,u32,i64,f64
"""000004CC6A9EA228A4D367C463A49B…",2024-08-29 00:00:00,1,1,0,0.0
"""00000D07B9A6256C0D44099CA726D3…",2025-04-24 00:00:00,1,1,0,0.0
"""000017720184E7316E75A7875EB214…",2025-07-03 00:00:00,2,2,2,1.0
"""00001E9887ECC2F2D27A2A8837939B…",2024-09-26 00:00:00,1,1,0,0.0
"""00002C7FD6E0080A69D0AB879C3D9B…",2024-11-21 00:00:00,1,1,0,0.0


In [23]:
import glob

parquet_files = sorted(glob.glob("/home/jovyan/Shared/2025-09-27-input/noteRating-*.parquet"))
print(f"Found {len(parquet_files)} files:")
for f in parquet_files[:5]:
    print(" ", f)


Found 0 files:


In [3]:
import os
import polars as pl
import duckdb
from datetime import datetime
from tqdm import tqdm
import gc

# =========================================================
# CONFIGURATION
# =========================================================
os.environ["POLARS_MAX_THREADS"] = "6"  # limit concurrency

INPUT_DIR   = "/home/jovyan/Shared/2025-09-27-input"
OUTPUT_DIR  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings"
os.makedirs(OUTPUT_DIR, exist_ok=True)

CUTOFF_DATE = datetime(2023, 1, 1)
BATCH_SIZE  = 2_000_000  # number of rows per DuckDB chunk

# =========================================================
# HELPER DEFINITIONS
# =========================================================
helpful_cols = [
    "helpfulOther", "helpfulClear", "helpfulGoodSources",
    "helpfulAddressesClaim", "helpfulImportantContext", "helpfulUnbiasedLanguage"
]
not_helpful_cols = [
    "notHelpfulIncorrect", "notHelpfulMissingKeyPoints",
    "notHelpfulHardToUnderstand", "notHelpfulArgumentativeOrBiased",
    "notHelpfulSpamHarassmentOrAbuse", "notHelpfulIrrelevantSources",
    "notHelpfulOpinionSpeculation", "notHelpfulNoteNotNeeded"
]
keep_cols = [
    "raterParticipantId", "createdAtMillis", "noteId", "ratedOnTweetId",
    "agree", "disagree", "helpfulnessLevel"
] + helpful_cols + not_helpful_cols

def safe_cast(df, cols, dtype=pl.Int64):
    for c in cols:
        if c in df.columns:
            df = df.with_columns(pl.col(c).cast(dtype, strict=False))
    return df

# =========================================================
# MAIN LOOP
# =========================================================
for i in tqdm(range(20), desc="Processing noteRatings files", dynamic_ncols=True):
    file_name = f"noteRatings-{i:05d}.parquet"
    input_path = os.path.join(INPUT_DIR, file_name)

    if not os.path.exists(input_path):
        print(f"⚠️  Skipping missing: {file_name}")
        continue

    print(f"\n📂 Processing {file_name} ...")
    con = duckdb.connect()
    total_rows = con.execute(f"SELECT COUNT(*) FROM '{input_path}'").fetchone()[0]
    print(f"  Rows in file: {total_rows:,}")

    chunk_results = []
    offset = 0
    batch_id = 0

    while offset < total_rows:
        batch_id += 1
        query = f"""
            SELECT {', '.join(keep_cols)}
            FROM '{input_path}'
            LIMIT {BATCH_SIZE} OFFSET {offset}
        """
        df = con.execute(query).pl()
        offset += BATCH_SIZE
        if df.is_empty():
            continue

        print(f"  🧩 Batch {batch_id} ({len(df):,} rows)")

        # --- Clean and transform ---
        df = df.rename({"raterParticipantId": "user_id", "createdAtMillis": "created_at_ms"})
        df = safe_cast(df, ["created_at_ms", "agree", "disagree"] + helpful_cols + not_helpful_cols, pl.Int64)

        df = df.with_columns(pl.from_epoch(pl.col("created_at_ms"), time_unit="ms").alias("created_at"))
        df = df.filter(pl.col("created_at") >= pl.lit(CUTOFF_DATE))

        if df.is_empty():
            continue

        df = df.with_columns(pl.col("created_at").dt.truncate("14d").alias("period_start"))

        df = df.with_columns([
            (pl.col("helpfulnessLevel") == "HELPFUL").cast(pl.Int8).alias("is_helpful"),
            (pl.col("helpfulnessLevel") == "SOMEWHAT_HELPFUL").cast(pl.Int8).alias("is_somewhat_helpful"),
            (pl.col("helpfulnessLevel") == "NOT_HELPFUL").cast(pl.Int8).alias("is_not_helpful")
        ])

        df = df.with_columns([
            pl.sum_horizontal([pl.col(c) for c in helpful_cols]).alias("helpful_flags_sum"),
            pl.sum_horizontal([pl.col(c) for c in not_helpful_cols]).alias("not_helpful_flags_sum")
        ])

        # --- Group and aggregate ---
        agg = (
            df.lazy()
            .group_by(["user_id", "period_start"])
            .agg([
                pl.len().alias("ratings_given"),
                pl.n_unique("noteId").alias("notes_rated_unique"),
                pl.n_unique("ratedOnTweetId").alias("tweets_rated_unique"),
                pl.mean("agree").alias("agree_rate"),
                pl.mean("disagree").alias("disagree_rate"),
                pl.mean("is_helpful").alias("helpful_rate"),
                pl.mean("is_somewhat_helpful").alias("somewhat_helpful_rate"),
                pl.mean("is_not_helpful").alias("not_helpful_rate"),
                pl.mean("helpful_flags_sum").alias("avg_helpful_flags"),
                pl.mean("not_helpful_flags_sum").alias("avg_not_helpful_flags")
            ])
            .sort(["user_id", "period_start"])
            .collect(streaming=True)
        )

        chunk_results.append(agg)

        del df, agg
        gc.collect()

    con.close()

    if len(chunk_results) == 0:
        print(f"⚠️  No valid rows in {file_name}")
        continue

    result = pl.concat(chunk_results, how="vertical_relaxed")
    out_path = os.path.join(OUTPUT_DIR, f"note_ratings_agg_{i:05d}.parquet")
    result.write_parquet(out_path)

    print(f"✅ Saved {out_path} ({result.height:,} rows)")
    del result, chunk_results
    gc.collect()

print("\n🎉 All noteRatings chunks processed safely and saved!")


Processing noteRatings files:   0%|          | 0/20 [00:00<?, ?it/s]


📂 Processing noteRatings-00000.parquet ...
  Rows in file: 14,115,918
  🧩 Batch 1 (2,000,000 rows)


/tmp/ipykernel_2687/1044297742.py:118: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)


  🧩 Batch 2 (2,000,000 rows)
  🧩 Batch 3 (2,000,000 rows)
  🧩 Batch 4 (2,000,000 rows)
  🧩 Batch 5 (2,000,000 rows)
  🧩 Batch 6 (2,000,000 rows)


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

...

✅ Saved /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings/note_ratings_agg_00017.parquet (340,530 rows)

📂 Processing noteRatings-00018.parquet ...
  Rows in file: 3,089,033
  🧩 Batch 1 (2,000,000 rows)
  🧩 Batch 2 (1,089,033 rows)


Processing noteRatings files:  95%|█████████▌| 19/20 [03:08<00:03,  3.39s/it]

✅ Saved /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings/note_ratings_agg_00018.parquet (335,706 rows)

📂 Processing noteRatings-00019.parquet ...
  Rows in file: 3,102,687
  🧩 Batch 1 (2,000,000 rows)
  🧩 Batch 2 (1,102,687 rows)


Processing noteRatings files: 100%|██████████| 20/20 [03:10<00:00,  9.54s/it]

✅ Saved /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings/note_ratings_agg_00019.parquet (342,156 rows)

🎉 All noteRatings chunks processed safely and saved!


In [2]:
import os
import polars as pl
import duckdb
from tqdm import tqdm
import gc
from datetime import datetime

# =========================================================
# CONFIGURATION
# =========================================================
os.environ["POLARS_MAX_THREADS"] = "6"

RATINGS_DIR = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings"
TEMP_DIR    = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings/tmp_merge"
FINAL_PATH  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_ratings_2023_aggregated.parquet"

os.makedirs(TEMP_DIR, exist_ok=True)
BATCH_SIZE  = 1_000_000  # adjustable
gc.collect()

# =========================================================
# FIND INPUT FILES
# =========================================================
files = sorted([
    os.path.join(RATINGS_DIR, f)
    for f in os.listdir(RATINGS_DIR)
    if f.endswith(".parquet") and f.startswith("note_ratings_agg_")
])

print(f"📦 Found {len(files)} aggregated rating files to merge")

# =========================================================
# STAGE 1: BATCHED INTERMEDIATE PARQUETS
# =========================================================
tmp_counter = 0
merged_batches = []

for fpath in tqdm(files, desc="Merging aggregated parquets", dynamic_ncols=True):
    print(f"\n📂 Reading {os.path.basename(fpath)} ...")

    con = duckdb.connect()
    total_rows = con.execute(f"SELECT COUNT(*) FROM '{fpath}'").fetchone()[0]
    print(f"  Rows in file: {total_rows:,}")

    offset = 0
    batch_id = 0

    while offset < total_rows:
        batch_id += 1
        query = f"SELECT * FROM '{fpath}' LIMIT {BATCH_SIZE} OFFSET {offset}"
        df = con.execute(query).pl()
        offset += BATCH_SIZE

        if df.is_empty():
            continue

        merged_batches.append(df)
        print(f"  🧩 Added batch {batch_id} ({len(df):,} rows)")

        # Flush every ~10 batches to temporary parquet
        if len(merged_batches) >= 10:
            combined = pl.concat(merged_batches, how="vertical_relaxed")
            tmp_path = os.path.join(TEMP_DIR, f"merged_part_{tmp_counter:03d}.parquet")
            combined.write_parquet(tmp_path)
            print(f"  💾 Flushed {combined.height:,} rows → {tmp_path}")
            tmp_counter += 1
            del merged_batches[:]
            del combined
            gc.collect()

    con.close()
    gc.collect()

# Final flush for remaining batches
if merged_batches:
    combined = pl.concat(merged_batches, how="vertical_relaxed")
    tmp_path = os.path.join(TEMP_DIR, f"merged_part_{tmp_counter:03d}.parquet")
    combined.write_parquet(tmp_path)
    print(f"✅ Final flush ({combined.height:,} rows) → {tmp_path}")
    tmp_counter += 1
    del merged_batches[:]
    gc.collect()

print(f"\n🧾 Wrote {tmp_counter} intermediate parquet chunks to {TEMP_DIR}")

# =========================================================
# STAGE 2: FINAL CONSOLIDATION WITH DUCKDB
# =========================================================
print("\n🚀 Consolidating all intermediate parts into final parquet...")

tmp_files = sorted([
    os.path.join(TEMP_DIR, f) for f in os.listdir(TEMP_DIR) if f.endswith(".parquet")
])

con = duckdb.connect()
query = f"""
COPY (
    SELECT * FROM read_parquet({tmp_files})
) TO '{FINAL_PATH}' (FORMAT PARQUET);
"""
con.execute(query)
con.close()

print(f"\n🎉 Successfully merged all aggregated files into:\n{FINAL_PATH}")


📦 Found 20 aggregated rating files to merge


Merging aggregated parquets:   0%|          | 0/20 [00:00<?, ?it/s]


📂 Reading note_ratings_agg_00000.parquet ...
  Rows in file: 1,720,180
  🧩 Added batch 1 (1,000,000 rows)


Merging aggregated parquets:   5%|▌         | 1/20 [00:01<00:21,  1.12s/it]

  🧩 Added batch 2 (720,180 rows)

📂 Reading note_ratings_agg_00001.parquet ...
  Rows in file: 1,724,554
  🧩 Added batch 1 (1,000,000 rows)


...

  🧩 Added batch 1 (335,706 rows)

📂 Reading note_ratings_agg_00019.parquet ...
  Rows in file: 342,156
  🧩 Added batch 1 (342,156 rows)


Merging aggregated parquets: 100%|██████████| 20/20 [00:23<00:00,  1.15s/it]

  💾 Flushed 3,513,507 rows → /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings/tmp_merge/merged_part_002.parquet

🧾 Wrote 3 intermediate parquet chunks to /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/ratings/tmp_merge

🚀 Consolidating all intermediate parts into final parquet...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))


🎉 Successfully merged all aggregated files into:
/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_ratings_2023_aggregated.parquet


In [ ]:
import os
import polars as pl
import duckdb
from datetime import datetime
from tqdm import tqdm
import gc

# =========================================================
# CONFIG
# =========================================================
os.environ["POLARS_MAX_THREADS"] = "6"

INPUT_PATH  = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_ratings_2023_aggregated.parquet"
TEMP_DIR    = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/tmp_reagg"
OUTPUT_PATH = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_ratings_2023_reaggregated.parquet"

os.makedirs(TEMP_DIR, exist_ok=True)
BATCH_SIZE  = 2_000_000  # adjust if needed
gc.collect()

# =========================================================
# STAGE 1 — PARTIAL REAGGREGATION PER BATCH
# =========================================================
con = duckdb.connect()
total_rows = con.execute(f"SELECT COUNT(*) FROM '{INPUT_PATH}'").fetchone()[0]
print(f"📦 Total rows in file: {total_rows:,}")

offset = 0
batch_id = 0
tmp_paths = []

while offset < total_rows:
    batch_id += 1
    query = f"SELECT * FROM '{INPUT_PATH}' LIMIT {BATCH_SIZE} OFFSET {offset}"
    df = con.execute(query).pl()
    offset += BATCH_SIZE

    if df.is_empty():
        continue

    print(f"🧩 Processing batch {batch_id} ({len(df):,} rows)")

    # --- Local groupby to shrink batch size ---
    agg = (
        df.lazy()
        .group_by(["user_id", "period_start"])
        .agg([
            pl.sum("ratings_given").alias("ratings_given"),
            pl.sum("notes_rated_unique").alias("notes_rated_unique"),
            pl.sum("tweets_rated_unique").alias("tweets_rated_unique"),
            pl.mean("agree_rate").alias("agree_rate"),
            pl.mean("disagree_rate").alias("disagree_rate"),
            pl.mean("helpful_rate").alias("helpful_rate"),
            pl.mean("somewhat_helpful_rate").alias("somewhat_helpful_rate"),
            pl.mean("not_helpful_rate").alias("not_helpful_rate"),
            pl.mean("avg_helpful_flags").alias("avg_helpful_flags"),
            pl.mean("avg_not_helpful_flags").alias("avg_not_helpful_flags")
        ])
        .sort(["user_id", "period_start"])
        .collect(streaming=True)
    )

    tmp_path = os.path.join(TEMP_DIR, f"reagg_part_{batch_id:03d}.parquet")
    agg.write_parquet(tmp_path)
    tmp_paths.append(tmp_path)

    print(f"  💾 Wrote partial reagg → {tmp_path} ({agg.height:,} rows)")

    del df, agg
    gc.collect()

con.close()

print(f"\n🧾 Wrote {len(tmp_paths)} temporary partial reaggregations to {TEMP_DIR}")

# =========================================================
# STAGE 2 — FINAL MERGE & GLOBAL REAGGREGATION
# =========================================================
print("\n🚀 Merging partial reaggregations into final dataset ...")

con = duckdb.connect()
query = f"""
COPY (
    SELECT
        user_id,
        period_start,
        SUM(ratings_given) AS ratings_given,
        SUM(notes_rated_unique) AS notes_rated_unique,
        SUM(tweets_rated_unique) AS tweets_rated_unique,
        AVG(agree_rate) AS agree_rate,
        AVG(disagree_rate) AS disagree_rate,
        AVG(helpful_rate) AS helpful_rate,
        AVG(somewhat_helpful_rate) AS somewhat_helpful_rate,
        AVG(not_helpful_rate) AS not_helpful_rate,
        AVG(avg_helpful_flags) AS avg_helpful_flags,
        AVG(avg_not_helpful_flags) AS avg_not_helpful_flags
    FROM read_parquet({tmp_paths})
    GROUP BY user_id, period_start
) TO '{OUTPUT_PATH}' (FORMAT PARQUET);
"""
con.execute(query)
con.close()

print(f"\n🎉 Global reaggregation complete!\nFinal file saved to:\n{OUTPUT_PATH}")

In [1]:
import os, gc, duckdb, polars as pl
from tqdm import tqdm

os.environ["POLARS_MAX_THREADS"] = "6"

TEMP_DIR    = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/tmp_reagg"
OUTPUT_PATH = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_ratings_2023_reaggregated.parquet"

# All partial parquet paths
tmp_files = sorted([
    os.path.join(TEMP_DIR, f) for f in os.listdir(TEMP_DIR)
    if f.endswith(".parquet")
])

print(f"🧾 Found {len(tmp_files)} partial aggregates")

# ---------------------------------------------------------
# 1️⃣  Incremental merge in small groups to limit memory
# ---------------------------------------------------------
INTERMEDIATE = os.path.join(TEMP_DIR, "stage2_parts")
os.makedirs(INTERMEDIATE, exist_ok=True)
group_size = 5               # merge 5 temp files at a time
stage2_parts = []

for i in tqdm(range(0, len(tmp_files), group_size), desc="Stage-2 partial merges"):
    subset = tmp_files[i:i+group_size]
    out_path = os.path.join(INTERMEDIATE, f"stage2_{i//group_size:03d}.parquet")

    con = duckdb.connect()
    # Force external processing and temp directory for spills
    con.execute(f"PRAGMA memory_limit='2GB';")
    con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")

    query = f"""
    COPY (
        SELECT
            user_id,
            period_start,
            SUM(ratings_given) AS ratings_given,
            SUM(notes_rated_unique) AS notes_rated_unique,
            SUM(tweets_rated_unique) AS tweets_rated_unique,
            AVG(agree_rate) AS agree_rate,
            AVG(disagree_rate) AS disagree_rate,
            AVG(helpful_rate) AS helpful_rate,
            AVG(somewhat_helpful_rate) AS somewhat_helpful_rate,
            AVG(not_helpful_rate) AS not_helpful_rate,
            AVG(avg_helpful_flags) AS avg_helpful_flags,
            AVG(avg_not_helpful_flags) AS avg_not_helpful_flags
        FROM read_parquet({subset})
        GROUP BY user_id, period_start
    ) TO '{out_path}' (FORMAT PARQUET);
    """
    con.execute(query)
    con.close()

    stage2_parts.append(out_path)
    gc.collect()

print(f"✅ Stage-2 wrote {len(stage2_parts)} mid-level files")

# ---------------------------------------------------------
# 2️⃣  Final consolidation (tiny set now)
# ---------------------------------------------------------
print("🚀 Running final external aggregation ...")
con = duckdb.connect()
con.execute(f"PRAGMA memory_limit='2GB';")
con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")

query = f"""
COPY (
    SELECT
        user_id,
        period_start,
        SUM(ratings_given) AS ratings_given,
        SUM(notes_rated_unique) AS notes_rated_unique,
        SUM(tweets_rated_unique) AS tweets_rated_unique,
        AVG(agree_rate) AS agree_rate,
        AVG(disagree_rate) AS disagree_rate,
        AVG(helpful_rate) AS helpful_rate,
        AVG(somewhat_helpful_rate) AS somewhat_helpful_rate,
        AVG(not_helpful_rate) AS not_helpful_rate,
        AVG(avg_helpful_flags) AS avg_helpful_flags,
        AVG(avg_not_helpful_flags) AS avg_not_helpful_flags
    FROM read_parquet({stage2_parts})
    GROUP BY user_id, period_start
) TO '{OUTPUT_PATH}' (FORMAT PARQUET);
"""
con.execute(query)
con.close()

print(f"🎉 Final reaggregation complete → {OUTPUT_PATH}")


🧾 Found 11 partial aggregates


Stage-2 partial merges:   0%|          | 0/3 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Stage-2 partial merges:  33%|███▎      | 1/3 [00:34<01:09, 34.55s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Stage-2 partial merges: 100%|██████████| 3/3 [01:02<00:00, 20.93s/it]


✅ Stage-2 wrote 3 mid-level files
🚀 Running final external aggregation ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🎉 Final reaggregation complete → /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/note_ratings_2023_reaggregated.parquet


In [5]:
import os
import polars as pl
import duckdb
from datetime import datetime
from tqdm import tqdm
import gc

# =========================================================
# CONFIG
# =========================================================
os.environ["POLARS_MAX_THREADS"] = "6"


INPUT_PATH = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/aggregates/note_ratings_2023_reaggregated.parquet"

con = duckdb.connect()
total_rows = con.execute(f"SELECT COUNT(1) FROM '{INPUT_PATH}'").fetchone()[0]
print(f"📦 Total rows in file: {total_rows:,}")

📦 Total rows in file: 20,869,319


In [2]:
import os, gc, duckdb
from tqdm import tqdm

os.environ["POLARS_MAX_THREADS"] = "6"

BASE_DIR = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/aggregates"
RATINGS  = os.path.join(BASE_DIR, "note_ratings_2023_reaggregated.parquet")
NOTES    = os.path.join(BASE_DIR, "notes_writers_2023_aggregated.parquet")
STATUS   = os.path.join(BASE_DIR, "note_status_history_2023_aggregated.parquet")
ENROLL   = os.path.join(BASE_DIR, "user_enrollment_2023_aggregated.parquet")
REQUESTS = os.path.join(BASE_DIR, "note_requests_2023_aggregated.parquet")

TEMP_DIR  = os.path.join(BASE_DIR, "super_merge_tmp2")
os.makedirs(TEMP_DIR, exist_ok=True)

MEM_LIMIT_GB = 2
BATCH_SIZE   = 2_000_000   # tune down if memory tight
part_files   = []

con = duckdb.connect()
total_rows = con.execute(f"SELECT COUNT(*) FROM '{RATINGS}'").fetchone()[0]
con.close()
print(f"Total ratings rows: {total_rows:,}")

offset = 0
batch_idx = 0

while offset < total_rows:
    batch_idx += 1
    out_path = os.path.join(TEMP_DIR, f"merged_part_{batch_idx:03d}.parquet")

    con = duckdb.connect()
    con.execute(f"PRAGMA memory_limit='{MEM_LIMIT_GB}GB';")
    con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")

    query = f"""
    COPY (
        WITH r AS (
            SELECT * FROM read_parquet('{RATINGS}')
            LIMIT {BATCH_SIZE} OFFSET {offset}
        )
        SELECT 
            r.*,
            n.* EXCLUDE (user_id, period_start),
            s.* EXCLUDE (user_id, period_start),
            e.* EXCLUDE (user_id, period_start),
            q.* EXCLUDE (user_id, period_start)
        FROM r
        LEFT JOIN read_parquet('{NOTES}')   n USING (user_id, period_start)
        LEFT JOIN read_parquet('{STATUS}')  s USING (user_id, period_start)
        LEFT JOIN read_parquet('{ENROLL}')  e USING (user_id, period_start)
        LEFT JOIN read_parquet('{REQUESTS}') q USING (user_id, period_start)
    ) TO '{out_path}' (FORMAT PARQUET);
    """
    print(f"🧩 Joining ratings batch {batch_idx} (offset {offset:,}) ...")
    con.execute(query)
    con.close()

    part_files.append(out_path)
    offset += BATCH_SIZE
    gc.collect()

print(f"✅ Wrote {len(part_files)} joined chunks to {TEMP_DIR}")

# -------- final consolidation (external mode) --------
print("🚀 Consolidating all joined parts...")
con = duckdb.connect()
con.execute(f"PRAGMA memory_limit='{MEM_LIMIT_GB}GB';")
con.execute(f"PRAGMA temp_directory='{TEMP_DIR}';")
final_path = os.path.join(BASE_DIR, "user_period_master.parquet")
con.execute(f"""
COPY (
  SELECT * FROM read_parquet({part_files})
) TO '{final_path}' (FORMAT PARQUET);
""")
con.close()

print(f"🎉 Master dataset ready → {final_path}")


Total ratings rows: 20,869,319
🧩 Joining ratings batch 1 (offset 0) ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🧩 Joining ratings batch 2 (offset 2,000,000) ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🧩 Joining ratings batch 3 (offset 4,000,000) ...


...

🧩 Joining ratings batch 11 (offset 20,000,000) ...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Wrote 11 joined chunks to /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/aggregates/super_merge_tmp2
🚀 Consolidating all joined parts...


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🎉 Master dataset ready → /home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/aggregates/user_period_master.parquet


In [1]:
import duckdb

path = "/home/jovyan/Shared/project1-group1/info-470-project-1/langdata/time-series-core/aggregates/user_period_master.parquet"

con = duckdb.connect()

# Read the parquet into DuckDB
df = con.execute(f"SELECT * FROM read_parquet('{path}') LIMIT 0").fetchdf()

# Show columns
print(df.columns)

Index(['user_id', 'period_start', 'ratings_given', 'notes_rated_unique',
       'tweets_rated_unique', 'agree_rate', 'disagree_rate', 'helpful_rate',
       'somewhat_helpful_rate', 'not_helpful_rate', 'avg_helpful_flags',
       'avg_not_helpful_flags', 'notes_written', 'num_misleading',
       'num_not_misleading', 'avg_trustworthySources', 'media_note_ratio',
       'misleading_flag_sum', 'not_misleading_flag_sum', 'share_misleading',
       'avg_flags_per_note', 'notes_with_status', 'notes_helpful',
       'notes_not_helpful', 'notes_nmr', 'notes_locked', 'first_helpful',
       'latest_helpful', 'latest_not_helpful', 'avg_days_to_first_nonNMR',
       'avg_days_to_lock', 'share_helpful', 'share_not_helpful', 'records',
       'avg_successfulRatingNeededToEarnIn', 'avg_days_since_state_change',
       'avg_days_since_earnout', 'is_new_user', 'is_earned_in', 'is_at_risk',
       'is_earned_out', 'has_ever_earned_out', 'is_core_population',
       'avg_modelingGroup', 'notes_requeste